Read in TSV file

In [5]:
def get_id(line):
    return "_".join(map(str, [line['chromosome_name'], line['start'], line['stop']]))

In [6]:
import pandas as pd
table = pd.read_table("A7.tsv")
table['id']=table.apply(get_id, axis=1)
table = table.set_index('id')
table.columns

Index([u'chromosome_name', u'start', u'stop', u'reference', u'variant',
       u'type', u'gene_name', u'transcript_name', u'transcript_species',
       u'transcript_source', u'transcript_version', u'strand',
       u'transcript_status', u'trv_type', u'c_position', u'amino_acid_change',
       u'ucsc_cons', u'domain', u'all_domains', u'deletion_substructures',
       u'transcript_error', u'brain.rcnt.llr3_ref', u'brain.rcnt.llr3_var',
       u'brain.rcnt.llr3_VAF', u'kidney.rcnt.llr3_ref',
       u'kidney.rcnt.llr3_var', u'kidney.rcnt.llr3_VAF',
       u'liver.rcnt.llr3_ref', u'liver.rcnt.llr3_var', u'liver.rcnt.llr3_VAF',
       u'lung.rcnt.llr3_ref', u'lung.rcnt.llr3_var', u'lung.rcnt.llr3_VAF',
       u'rib.rcnt.llr3_ref', u'rib.rcnt.llr3_var', u'rib.rcnt.llr3_VAF',
       u'tumor.rcnt.llr3_ref', u'tumor.rcnt.llr3_var', u'tumor.rcnt.llr3_VAF',
       u'cluster'],
      dtype='object')

In [7]:
ref_cols = ['tumor.rcnt.llr3_ref','brain.rcnt.llr3_ref', 'kidney.rcnt.llr3_ref', 'liver.rcnt.llr3_ref', 'lung.rcnt.llr3_ref', 'rib.rcnt.llr3_ref']
var_cols = ['tumor.rcnt.llr3_var','brain.rcnt.llr3_var', 'kidney.rcnt.llr3_var', 'liver.rcnt.llr3_var', 'lung.rcnt.llr3_var', 'rib.rcnt.llr3_var']

#breast,adrenal,liver,lung,spinal
cols = ['breast', 'brain', 'kidney', 'liver', 'lung', 'rib']
table = table[['cluster']+ref_cols+var_cols]
table.columns = ['cluster']+['ref-'+c for c in cols] + ['var-'+c for c in cols]
table.head()

,cluster,ref-breast,ref-brain,ref-kidney,ref-liver,ref-lung,ref-rib,var-breast,var-brain,var-kidney,var-liver,var-lung,var-rib
id,,,,,,,,,,,,,
1_2866642_2866642,1,125,131,169,110,114,102,42,96,77,79,57,85
1_2984058_2984058,3,205,211,239,135,176,212,0,1,43,87,1,0
1_3814510_3814510,6,170,189,242,185,150,82,0,0,1,0,3,58
1_4293949_4293949,1,100,85,138,70,69,51,45,43,38,64,27,45
1_4367117_4367117,9,187,206,227,170,173,179,0,0,0,25,0,0


In [8]:
ctable = table.groupby('cluster').mean()

Get intervals

In [9]:
import numpy
from scipy.stats import beta
from scipy.stats import norm

def binomial_hpdr(n, N, pct, a=1, b=1, n_pbins=1e3):
    """
    Function computes the posterior mode along with the upper and lower bounds of the
    **Highest Posterior Density Region**.

    Parameters
    ----------
    n: number of successes 
    N: sample size 
    pct: the size of the confidence interval (between 0 and 1)
    a: the alpha hyper-parameter for the Beta distribution used as a prior (Default=1)
    b: the beta hyper-parameter for the Beta distribution used as a prior (Default=1)
    n_pbins: the number of bins to segment the p_range into (Default=1e3)

    Returns
    -------
    A tuple that contains the mode as well as the lower and upper bounds of the interval
    (mode, lower, upper)

    """
    # fixed random variable object for posterior Beta distribution
    rv = beta(n+a, N-n+b)
    # determine the mode and standard deviation of the posterior
    stdev = rv.stats('v')**0.5
    mode = (n+a-1.)/(N+a+b-2.)
    # compute the number of sigma that corresponds to this confidence
    # this is used to set the rough range of possible success probabilities
    n_sigma = numpy.ceil(norm.ppf( (1+pct)/2. ))+1
    # set the min and max values for success probability 
    max_p = mode + n_sigma * stdev
    if max_p > 1:
        max_p = 1.
    min_p = mode - n_sigma * stdev
    if min_p > 1:
        min_p = 1.
    # make the range of success probabilities
    p_range = numpy.linspace(min_p, max_p, n_pbins+1)
    # construct the probability mass function over the given range
    if mode > 0.5:
        sf = rv.sf(p_range)
        pmf = sf[:-1] - sf[1:]
    else:
        cdf = rv.cdf(p_range)
        pmf = cdf[1:] - cdf[:-1]
    # find the upper and lower bounds of the interval 
    sorted_idxs = numpy.argsort( pmf )[::-1]
    cumsum = numpy.cumsum( numpy.sort(pmf)[::-1] )
    j = numpy.argmin( numpy.abs(cumsum - pct) )
    upper = p_range[ (sorted_idxs[:j+1]).max()+1 ]
    lower = p_range[ (sorted_idxs[:j+1]).min() ]    

    return (mode, lower, upper)

In [12]:


def get_ub(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], 0.95)
    return v[2]
    

def get_lb(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], 0.95)
    mval = v[1]
    #if mval < 0.01: mval = 0
    return mval

def get_mean(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], 0.95)
    mval = v[0]
    return mval

ctable = table.groupby('cluster').mean()
for sam in cols:
    ctable['ub-'+sam]= ctable.apply(get_ub, args=[sam], axis=1)
    ctable['lb-'+sam]= ctable.apply(get_lb, args=[sam], axis=1)
    ctable[sam]= ctable.apply(get_mean, args=[sam], axis=1)
    

In [13]:
ctable

,ref-breast,ref-brain,ref-kidney,ref-liver,ref-lung,ref-rib,var-breast,var-brain,var-kidney,var-liver,...,kidney,ub-liver,lb-liver,liver,ub-lung,lb-lung,lung,ub-rib,lb-rib,rib
cluster,,,,,,,,,,,,,,,,,,,,,
1,122.435583,150.245399,214.073620,110.092025,105.527607,115.073620,43.496933,72.957055,57.699387,79.674847,...,0.212307,0.490472,3.513744e-01,0.419856,0.423257,0.277947,0.348151,0.436810,2.979442e-01,0.365460
2,165.066667,208.200000,199.466667,100.866667,152.133333,106.800000,0.266667,1.933333,53.866667,78.933333,...,0.212632,0.511694,3.680804e-01,0.439006,0.053175,0.005690,0.021441,0.435827,2.916587e-01,0.361753
3,168.027027,222.405405,209.000000,101.864865,162.675676,175.270270,0.135135,2.108108,55.621622,82.945946,...,0.210193,0.520684,3.784718e-01,0.448815,0.019674,-0.000018,0.000830,0.047685,5.379217e-03,0.019504
4,159.071429,141.380952,258.690476,187.476190,104.071429,172.833333,0.119048,67.285714,0.309524,0.095238,...,0.001195,0.016715,-1.930711e-05,0.000508,0.401796,0.255384,0.325671,0.019366,-2.660135e-07,0.001238
5,160.100000,206.400000,189.400000,99.600000,148.300000,162.000000,0.100000,0.000000,49.000000,63.100000,...,0.205537,0.463735,3.155614e-01,0.387830,0.021116,-0.000032,0.000674,0.019364,-3.084801e-05,0.000617
6,170.087912,225.252747,269.681319,199.351648,161.945055,115.186813,0.076923,0.054945,0.186813,0.109890,...,0.000692,0.015876,-1.088565e-05,0.000551,0.041130,0.002583,0.014050,0.415486,2.762537e-01,0.343521
7,162.363636,227.545455,285.727273,212.545455,108.090909,173.727273,0.000000,0.272727,0.181818,0.272727,...,0.000636,0.016264,-3.584454e-06,0.001282,0.367887,0.225118,0.293103,0.018918,-2.318666e-05,0.001045
8,139.416667,185.708333,173.250000,169.000000,135.750000,148.166667,0.166667,0.041667,46.041667,0.125000,...,0.209956,0.018849,-7.544080e-08,0.000739,0.022782,-0.000016,0.000613,0.020406,-2.308477e-06,0.000281
9,168.325000,226.212500,274.100000,156.200000,164.837500,184.825000,0.062500,0.350000,0.150000,36.075000,...,0.000547,0.246507,1.368359e-01,0.187622,0.018464,-0.000026,0.000303,0.016500,-2.402629e-05,0.000270


In [14]:
def get_ub(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], 0.95)
    return v[2]
    

def get_lb(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], 0.95)
    mval = v[1]
    if mval < 0.01: mval = 0
    return mval

def get_mean(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], 0.95)
    mval = v[0]
    return mval

ctable_cutoff = table.groupby('cluster').mean()
for sam in cols:
    ctable_cutoff['ub-'+sam]= ctable.apply(get_ub, args=[sam], axis=1)
    ctable_cutoff['lb-'+sam]= ctable.apply(get_lb, args=[sam], axis=1)
    ctable_cutoff[sam]= ctable.apply(get_mean, args=[sam], axis=1)

In [15]:

def get_ub(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], 0.95)
    return v[2]
    

def get_lb(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], 0.95)
    mval = v[1]
    if mval < 0.01: mval = 0
    return mval

def get_vaf(row, sam):
    return float(row['var-'+sam])/float(row['var-'+sam]+row['ref-'+sam])

#ctable_cutoff = table.groupby('cluster').mean()
vafs = pd.DataFrame()
for sam in cols:
    vafs[sam] = table.apply(get_vaf, args=[sam], axis=1)
vafs['cluster'] = table['cluster']
stds = vafs.groupby('cluster').std()

stds.columns = ['std-'+sam for sam in cols]
print vafs.groupby('cluster').mean().head()
print stds.head()

           breast     brain    kidney     liver      lung       rib
cluster                                                            
1        0.260448  0.327485  0.212014  0.418261  0.348278  0.365536
2        0.001359  0.008929  0.211260  0.438547  0.022799  0.361575
3        0.000657  0.009655  0.210790  0.451779  0.000905  0.017655
4        0.000707  0.321041  0.001203  0.000473  0.326051  0.001396
5        0.000758  0.000000  0.205312  0.390257  0.000909  0.000420
         std-breast  std-brain  std-kidney  std-liver  std-lung   std-rib
cluster                                                                  
1          0.045521   0.041771    0.031925   0.080755  0.049325  0.045551
2          0.004133   0.007712    0.030785   0.065276  0.011218  0.040714
3          0.001694   0.006476    0.030714   0.055228  0.002847  0.074728
4          0.002283   0.037253    0.002523   0.001542  0.038454  0.003361
5          0.002396   0.000000    0.018411   0.073778  0.002875  0.001329


In [16]:
cvafs = vafs.groupby('cluster').mean()
cvafs = pd.merge(cvafs, stds, left_index=True, right_index=True)


def get_ub(row, sam):
    return min(row[sam]+2*(row['std-'+sam]), 1.)

def get_lb(row, sam):
    if row[sam] < 0.01: return max(row[sam]-2*(row['std-'+sam]), 0)
    else: return max(row[sam]-2*(row['std-'+sam]), 0.01)

for sam in cols:
    cvafs['ub-'+sam] = cvafs.apply(get_ub, args=[sam], axis=1)
    cvafs['lb-'+sam] = cvafs.apply(get_lb, args=[sam], axis=1)


In [17]:
cvafs

,breast,brain,kidney,liver,lung,rib,std-breast,std-brain,std-kidney,std-liver,...,ub-brain,lb-brain,ub-kidney,lb-kidney,ub-liver,lb-liver,ub-lung,lb-lung,ub-rib,lb-rib
cluster,,,,,,,,,,,,,,,,,,,,,
1,0.260448,0.327485,0.212014,0.418261,0.348278,0.365536,0.045521,0.041771,0.031925,0.080755,...,0.411026,0.243943,0.275864,0.148164,0.579771,0.256751,0.446928,0.249627,0.456638,0.274435
2,0.001359,0.008929,0.211260,0.438547,0.022799,0.361575,0.004133,0.007712,0.030785,0.065276,...,0.024352,0.000000,0.272831,0.149689,0.569098,0.307996,0.045234,0.010000,0.443002,0.280148
3,0.000657,0.009655,0.210790,0.451779,0.000905,0.017655,0.001694,0.006476,0.030714,0.055228,...,0.022606,0.000000,0.272218,0.149361,0.562236,0.341322,0.006600,0.000000,0.167111,0.010000
4,0.000707,0.321041,0.001203,0.000473,0.326051,0.001396,0.002283,0.037253,0.002523,0.001542,...,0.395548,0.246535,0.006248,0.000000,0.003556,0.000000,0.402960,0.249143,0.008119,0.000000
5,0.000758,0.000000,0.205312,0.390257,0.000909,0.000420,0.002396,0.000000,0.018411,0.073778,...,0.000000,0.000000,0.242133,0.168490,0.537813,0.242700,0.006659,0.000000,0.003078,0.000000
6,0.000507,0.000237,0.000719,0.000529,0.014153,0.344677,0.001858,0.001017,0.001943,0.001555,...,0.002271,0.000000,0.004606,0.000000,0.003638,0.000000,0.041625,0.010000,0.476265,0.213090
7,0.000000,0.001125,0.000559,0.001077,0.290363,0.001358,0.000000,0.001964,0.001247,0.002500,...,0.005052,0.000000,0.003053,0.000000,0.006078,0.000000,0.368394,0.212332,0.007459,0.000000
8,0.001367,0.000230,0.209567,0.000699,0.000798,0.000296,0.004553,0.001128,0.029517,0.001902,...,0.002486,0.000000,0.268600,0.150533,0.004504,0.000000,0.006203,0.000000,0.003191,0.000000
9,0.000365,0.001648,0.000454,0.191827,0.000223,0.000275,0.001484,0.003171,0.001490,0.067023,...,0.007990,0.000000,0.003433,0.000000,0.325872,0.057782,0.002615,0.000000,0.002713,0.000000


In [18]:


rows = ["6 #m\n10 #n\n#sample_index\tsample_label\tcharacter_label\tcharacter_index\tvaf_lb\tvaf_mean\tvaf_ub\tx\ty\tmu\n",]

def print_char(row, sam):
    #print "\t".join(map(str,[i, sam, row.name-1, 'cluster_'+str(row.name), row['lb-'+sam], row[sam], row['lb-'+sam], 1, 1, 1]))
    return "\t".join(map(str,[i, sam, row.name-1, 'cluster_'+str(row.name), row['lb-'+sam], row[sam], row['ub-'+sam], 1, 1, 1]))+"\n"

for i, sam in enumerate(cols):
    rows += list(cvafs.apply(print_char, args=[sam], axis=1))

with open("A7_spruce_vafs.tsv", 'w') as f:
    for line in rows:
        f.write(line)

        



In [19]:
rows = ["6 #m\n10 #n\n#sample_index\tsample_label\tcharacter_label\tcharacter_index\tvaf_lb\tvaf_mean\tvaf_ub\tx\ty\tmu\n",]

def print_char(row, sam):
    #print "\t".join(map(str,[i, sam, row.name-1, 'cluster_'+str(row.name), row['lb-'+sam], row[sam], row['lb-'+sam], 1, 1, 1]))
    return "\t".join(map(str,[i, sam, row.name-1, 'cluster_'+str(row.name), row['lb-'+sam], row[sam], row['ub-'+sam], 1, 1, 1]))+"\n"

for i, sam in enumerate(cols):
    rows += list(ctable_cutoff.apply(print_char, args=[sam], axis=1))

with open("A7_spruce_betas.tsv", 'w') as f:
    for line in rows:
        f.write(line)


In [20]:
rows = ["6 #m\n10 #n\n#sample_index\tsample_label\tcharacter_label\tcharacter_index\tvaf_lb\tvaf_mean\tvaf_ub\tx\ty\tmu\n",]

def print_char(row, sam):
    #print "\t".join(map(str,[i, sam, row.name-1, 'cluster_'+str(row.name), row['lb-'+sam], row[sam], row['lb-'+sam], 1, 1, 1]))
    return "\t".join(map(str,[i, sam, row.name-1, 'cluster_'+str(row.name), row['lb-'+sam], row[sam], row['ub-'+sam], 1, 1, 1]))+"\n"

for i, sam in enumerate(cols):
    rows += list(ctable.apply(print_char, args=[sam], axis=1))

with open("A7_spruce_betas_nocutoff.tsv", 'w') as f:
    for line in rows:
        f.write(line)
